#### ¿Hacia donde se dirige el valor de las acciones en 2026?
Al nacer la necesidad financiera de descifrar la trayectoria en el mercado y el sentimiento que despiertan los activos, el objetivo es identificar cuándo una acción está ganando fuerza real y cuándo su valor es vulnerable, proporcionando una base sólida para maximizar la rentabilidad y gestionar el riesgo con precisión para el primer trimestre de 2026.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
import pandas_ta as ta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input


df = pd.read_csv("data/stock_market.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

#### Análisis Exploratorio de Series Temporales (OHLC, Volume)
Aborda las variables fundamentales del mercado desde su naturaleza secuencial y cronológica. Se enfoca en descomponer los movimientos de los precios OHLC y el volumen de intercambio para identificar patrones de tendencia a largo plazo y ciclos de estacionalidad ocultos. Es vital entender si el mercado en general ha estado en una tendencia alcista o bajista a largo plazo, y si hay patrones que se repiten.

#### Análisis Exploratorio Tradicional (RSI, MACD, Sentiment)
Tiene su enfoque sobre los indicadores derivados de los datos de series temporales, cuya naturaleza matemática los hace aptos de un análisis estadístico más clásico (Distribuciones, Medidas, Correlaciones). El objetivo es validar la calidad de las señales derivadas y comprender su comportamiento intrínseco y su relación con el mercado.

In [ ]:
df.set_index("Date", inplace=True)

fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 20), sharex=True)

variables = ["Open","High","Low","Close","Volume"]
colors = ["#1f77b4","#ff7f0e","#2ca02c","#d62728","#7f7f7f"]

for i, var in enumerate(variables):
    axs[i].plot(df.index, df[var], label=f"{var} Real", color=colors[i], alpha=0.3)
    
    trend = df[var].rolling(window=30).mean()
    axs[i].plot(df.index, trend, label="Tendencia (SMA 30)", color=colors[i], linewidth=2)
    
    axs[i].set_title(f"Análisis de Tendencia: {var}", fontsize=14)
    axs[i].legend(loc="upper left")
    axs[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Tendencia y Estacionariedad evidente
El análisis revela que las variables 'OHLC' presentan una tendencia clara y persistente (comportamiento no estacionario), mientras que la variable 'Volume' muestra una evidente estacionariedad, caracterizada por una reversión constante a su media sin una dirección de crecimiento definida a largo plazo.

Mientras que el precio puede subir infinitamente (tendencia creciente), el volumen de intercambio humano y algorítmico suele tener límites físicos y temporales. La estacionariedad en el volumen sugiere que el interés en el activo es constante a largo plazo, pero con ráfagas de actividad.

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(16, 25))
variables = ["Open","High","Low","Close","Volume"]

for i, var in enumerate(variables):
    data_clean = df[var].dropna()
    plot_acf(data_clean, ax=axs[i], lags=140, title=f"Correlograma de Estacionalidad: {var}")
    
    axs[i].set_ylabel("Autocorrelación")
    axs[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Adiós a la Estacionalidad, Hola a la Volatilidad
La alta autocorrelación en los primeros lags que baja lentamente sin picos en las variables OHLC indica que el precio tiene "memoria" de corto plazo, pero no hay un patrón cíclico (estacionalidad) que se repita. Esto significa que los precios son No Estacionarios y siguen un proceso de Caminata Aleatoria (Random Walk), los cambios de precio de hoy no dependen de los cambios de precio de ayer.

En cuanto al volumen de intercambios, su correlograma muestra barras muy cortas que suben y bajan alrededor del cero sin un orden claro, lo que significa que se comporta como "ruido" o eventos independientes. Sin una estacionalidad marcada, se confirma que los eventos de intercambio masivo son esporádicos y probablemente impulsados por noticias o shocks externos, no por ciclos calendarios.

In [ ]:
df["RSI"] = ta.rsi(df["Close"], length=14)

macd = df.ta.macd(close="Close", fast=12, slow=26, signal=9)
df = pd.concat([df, macd], axis=1)

df["EMA_12"] = df["Close"].ewm(span=12, adjust=False).mean()
df["EMA_26"] = df["Close"].ewm(span=26, adjust=False).mean()

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05, 
                    subplot_titles=("Precio (EMA)","RSI (Zonas de Reacción)","Sentimiento vs. Volumen"),
                    row_heights=[0.5, 0.25, 0.25])

# Nivel 1: Precio y MACD (Impulso)
fig.add_trace(go.Scatter(x=df.index, y=df["Close"], name="Precio"), row=1, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA_12"], name="EMA 12 (Rápida)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA_26"], name="EMA 26 (Lenta)"), row=1, col=1)

# Nivel 2: RSI con Bandas 70/30
fig.add_trace(go.Scatter(x=df.index, y=df["RSI"], name="RSI", line=dict(color="purple")), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

# Nivel 3: Sentimiento vs. Volumen (Correlación)
fig.add_trace(go.Scatter(x=df["Sentiment"], y=df["Volume"], mode="markers",
        marker=dict(
            size=8,
            color=df["Close"],  
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Precio", x=1.05)
        ),
        name="Sentimiento vs Volumen"
    ), row=3, col=1)

fig.update_layout(height=900, title_text="EDA Avanzado: Factores Externos", showlegend=False)
fig.show()

# 3. Análisis de Correlación de Spearman (Relación no lineal)
plt.figure(figsize=(15, 6))
correlation_matrix = df[["Close","RSI","MACD_12_26_9","Sentiment"]].corr(method="spearman")
sns.heatmap(correlation_matrix, annot=True, cmap="RdYlGn", center=0)
plt.title("Matriz de Correlación")
plt.show()

#### Modelado de Random Forest ( junto con el método de Monte Carlo)
La premisa de que los insights críticos no residen en la inercia cronológica de la serie, como precios pasados o volumenes, sino en las señales exógenas y de momento capturadas demostró que el comportamiento del activo está dominado por la volatilidad dinámica y no por patrones estacionales o ciclos crónicos previsibles. Esta deducción obligó a abandonar los modelos lineales convencionales y seleccionar a un meta-algoritmo, siendo este último excepcional capacidad para mapear relaciones no lineales entre estos indicadores heterogéneos, integrándolo con otra técnica para generar miles de trayectorias posibles.

In [74]:
df["Close_yesterday"] = df["Close"].shift(1)
df_ml = df[["Close","Close_yesterday","RSI","MACD","Sentiment"]].dropna()

X = df_ml[["Close_yesterday","RSI","MACD","Sentiment"]]
y = df_ml["Close"]

forecast_model = RandomForestRegressor(n_estimators=100, random_state=42)
forecast_model.fit(X, y)

random_sample = df_ml.sample(frac=0.3, random_state=42)

X_sample = random_sample[["Close_yesterday", "RSI", "MACD", "Sentiment"]]
y_sample = random_sample["Close"]

sample_predicts = forecast_model.predict(X_sample)

MAE = mean_absolute_error(y_sample, sample_predicts)
MAE_ = round(MAE, 2)
MAE_ = "$"+str(MAE_)

#### Dashboard que traza un análisis de tendencia mensual junto con diferentes pronósticos en las acciones

In [ ]:
df.reset_index(inplace=True)

max_month_mean = html.B(children=[], id="max")
min_month_mean = html.B(children=[], id="min")

dates = [
    {"label":"Primer Mes","value":30},
    {"label":"Primer Trimestre","value":90},
    {"label":"Primer Cuatrimestre","value":120},
    {"label":"Primer Semestre","value":180}
]

html_b = html.B("Acciones ($)", style={"color":"yellow"})

MAE = html.B(MAE_, id="MAE")
ROI = html.B(children=[], id="ROI")
STD = html.B(children=[], id="STD")

app = dash.Dash(__name__)

app.layout =  html.Div(id="body", className="e7_body", children=[
    html.H1("Análisis de Tendencia Mensual", id="H1", className="e7_title"),
    html.Div(id="KPI_div_1", className="e7_KPI_div_1", children=[
        html.P(max_month_mean, className="e7_KPI_1"),
        html.P(min_month_mean, className="e7_KPI_1")
    ]),
    html.Div(id="graph_div_1", className="e7_graph_div_1", children=[
       dcc.Dropdown(id="dropdown_1", className="e7_dropdown_1",
                        options=[
                            {"label":"Volumen","value":"Volume"},
                            {"label":"Precio ($)","value":"Close"}
                        ],
                        value="Volume",
                        multi=False,
                        clearable=False),
        dcc.Graph(id="trend_analysis", figure={}, className="e7_graph_1")
    ]),
    html.H2(["Pronóstico de ", html_b], id="H2", className="e7_title"),
    html.Div(id="forecast_div", className="e7_forecast_div", children=[
        html.Div(id="KPI_div_2", className="e7_KPI_div_2", children=[
            html.Div(className="e7_KPI_2", children=[html.P("MAE", className="e7_KPI_title"), html.P(MAE, className="e7_KPI_p")]),
            html.Div(className="e7_KPI_2", children=[html.P("ROI", className="e7_KPI_title"), html.P(ROI, className="e7_KPI_p")]),
            html.Div(className="e7_KPI_2", children=[html.P("STD", className="e7_KPI_title"), html.P(STD, className="e7_KPI_p")])
        ]),
        html.Div(id="graph_div_2", className="e7_graph_div_2", children=[
            html.Div(id="dropdown_div", className="e7_dropdown_div", children=[
                dcc.Dropdown(id="dropdown_2", className="e7_dropdown_2",
                        options=dates,
                        value=30,
                        multi=False,
                        clearable=False)]),
            dcc.Graph(id="forecasting", figure={}, className="e7_graph_2")    
        ])
    ])
])


@app.callback(
    [Output(component_id="max",component_property="children"),
    Output(component_id="min",component_property="children"),
    Output(component_id="trend_analysis",component_property="figure"),
    Output(component_id="forecasting",component_property="figure"),
    Output(component_id="ROI",component_property="children"),
    Output(component_id="STD",component_property="children")],
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

def update_graph(slct_var, slct_days):
    
    df_seg = df.groupby(pd.Grouper(key="Date", freq="ME"))[slct_var].mean().reset_index()
    df_seg["month"] = df_seg["Date"].dt.strftime("%b")
    df_seg["month_year"] = df_seg["Date"].dt.strftime("%b %Y")

    month_mean = df_seg.groupby("month")[slct_var].mean().reset_index()

    max_mean = str(round(month_mean.max()[1], 1))
    max_month = month_mean.max()[0]
    max_month_mean = f"Máx: {max_month} ({max_mean}$)"

    min_mean = str(round(month_mean.min()[1], 1))
    min_month = month_mean.min()[0]
    min_month_mean = f"Mín: {min_month} ({min_mean}$)"
    
    months_means = go.Figure()
    months_means.add_trace(go.Scatter(x=df_seg["month_year"], y=df_seg[slct_var], mode="lines", fill="tozeroy", fillcolor="rgba(0, 255, 0, 0.2)"))
    months_means.update_layout(title_text="Promedios mensuales",  xaxis_title=" ", yaxis_title=" ")
    
    label_slct = next(
        (option["label"] for option in dates if option["value"] == slct_days), 
        slct_days
    )
    
    last_row = df_ml.iloc[-1]
    
    actual_close = last_row["Close"]
    actual_rsi = last_row["RSI"]
    actual_macd = last_row["MACD"]
    actual_sentiment = last_row["Sentiment"]
    actual_date = pd.to_datetime(df["Date"]).max()
    
    predicts = []
    future_dates = []

    volatility = df["Close"].pct_change().std()  

    for _ in range(slct_days):
        input_data = [[actual_close, actual_rsi, actual_macd, actual_sentiment]]
        
        price_predict = forecast_model.predict(input_data)[0]
        variability = np.random.normal(0, volatility) 
        price_predict = price_predict * (1 + variability)
        predicts.append(price_predict)
        
        actual_date += pd.Timedelta(days=1)
        future_dates.append(actual_date)
        
        actual_close = price_predict 
        
    df_forecast = pd.DataFrame({
        "Date":future_dates,
        "Forecast":predicts
    })
    
    last_five_years = datetime.now() - timedelta(days=2*365) 
    df_fil = df[df["Date"] >= last_five_years].copy()
    
    forecasting = go.Figure()
    forecasting.add_trace(go.Scatter(x=df_fil["Date"], y=df_fil["Close"], mode="lines", fill="tozeroy", fillcolor="rgba(0, 0, 255, 0.2)", name="Período 2024-2025"))
    forecasting.add_trace(go.Scatter(x=df_forecast["Date"], y=df_forecast["Forecast"], name="Pronóstico 2026", fill="tozeroy", fillcolor="rgba(255, 165, 0, 0.2)"))
    forecasting.update_layout(title_text=f"Pronóstico del {label_slct}",  xaxis_title=" ", yaxis_title=" ")
    
    future_prices = np.array(predicts).flatten()
    initial_price = df["Close"].iloc[-1]
    final_price = future_prices[-1]
    ROI = round(((final_price - initial_price) / initial_price) * 100, 2)
    if ROI > 0:
        ROI = "+"+str(ROI)+"%"
    else:
        ROI = str(ROI)+"%"

    STDs = np.diff(np.insert(future_prices, 0, initial_price)) / np.insert(future_prices, 0, initial_price)[:-1]
    STD = round(np.std(STDs) * 100, 2)
    STD = "$"+str(STD)

    return max_month_mean, min_month_mean, months_means, forecasting, ROI, STD

if __name__ == "__main__":
    app.run_server(debug=False)

#### Conclusión final: Más allá del espejo del precio
Al reconocer la incertidumbre inherente al mercado, se la transforma en un mapa de probabilidades donde la volatilidad ya no es el enemigo, sino el lenguaje mismo de la oportunidad y una herramienta de gestión de riesgo accionable.